In [1]:
import numpy as np
import pandas as pd
import os
from scipy.stats import norm
pd.set_option('display.max_columns', 500)

In [2]:
path = '/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Options/Datasets/MR-PYT-065 - Index Option.xlsx'

In [3]:
#reading the input data
rates_interest = pd.read_excel(path,sheet_name = 'rates_interest')
rates_instruments = pd.read_excel(path,sheet_name = 'rates_instruments')
rates_implied_volatility = pd.read_excel(path,sheet_name = 'rates_implied_volatility')
rates_exchange = pd.read_excel(path,sheet_name = 'rates_exchange')
exposure = pd.read_excel(path,sheet_name = 'Exposure')

#output_1 = pd.read_excel(path,sheet_name = 'OUTPUT 1')
#output_2 = pd.read_excel(path,sheet_name = 'OUTPUT 2')

In [4]:
output_1 = exposure.copy(deep = True)

mapping_dict_instruments = dict(zip(rates_instruments['instrument_code'], rates_instruments['price']))

# Map the values from df1 to df2 based on column E
output_1['underlying_spot_rate'] = output_1['underlying_instrument_code'].map(mapping_dict_instruments)

output_1['residual_maturity_days'] = (output_1['Contract expiry date'] - output_1['As_On_Date']).dt.days

# Create a mapping dictionary for columns A and B in df1
mapping_dict_interest_rates = dict(zip(zip(rates_interest['curve'], rates_interest['tenor']), rates_interest['rate']))

output_1['lower_tenor_rate'] = output_1.apply(lambda row: mapping_dict_interest_rates.get((row['risk_free_curve'], row['lower_tenor']), 0), axis=1)
output_1['upper_tenor_rate'] = output_1.apply(lambda row: mapping_dict_interest_rates.get((row['risk_free_curve'], row['upper_tenor']), 0), axis=1)

In [7]:
condition = output_1['lower_tenor'] == output_1['upper_tenor']
output_1['interpolated_rate'] = np.where(condition, output_1['lower_tenor_rate'],
                                         ((output_1['upper_tenor_rate'] - output_1['lower_tenor_rate']) * 
                                          (output_1['upper_maturity_days'] - output_1['lower_maturity_days']) * 
                                          (output_1['residual_maturity_days'] - output_1['lower_maturity_days']) + 
                                          output_1['lower_tenor_rate']))

In [8]:
# Create a mapping dictionary for columns A and B in df1
mapping_dict_rates_implied_volatility = dict(zip(zip(rates_implied_volatility['underlying_instrument_code'], rates_implied_volatility['date']), rates_implied_volatility['implied_volatility']))
output_1['implied_volatility'] = output_1.apply(lambda row: mapping_dict_rates_implied_volatility.get((row['underlying_instrument_code'], row['As_On_Date']), 0), axis=1)

In [9]:
output_1.columns

Index(['As_On_Date', 'Transaction_No', 'Exchange', 'entity_code',
       'product_code', 'dealer_code', 'underlying_instrument_code',
       'Underlying Asset', 'option_style', 'option_type',
       'currency_transaction', 'currency_domestic', 'currency_reporting',
       'Buy_Sell', 'Lot Size', 'No of contracts', 'Strike Price',
       'book_value_transaction_currency', 'book_value_domestic_currency',
       'book_value_reporting_currency', 'contract_cycle',
       'Contract expiry date', 'residual_maturity_yrs',
       'total_dividend_received_during_contract_cycle', 'dividend_yield',
       'risk_free_curve', 'lower_tenor', 'upper_tenor', 'lower_maturity_days',
       'upper_maturity_days', 'Exotic_Category', 'Accounting_Book',
       'Accounting_Portfolio', 'MTM_method', 'underlying_spot_rate',
       'residual_maturity_days', 'lower_tenor_rate', 'upper_tenor_rate',
       'interpolated_rate', 'implied_volatility'],
      dtype='object')

In [10]:
output_1['PV(total_dividend_received_during_contract_cycle)'] = 0
output_1['d1'] = (np.log((output_1['underlying_spot_rate']-output_1['PV(total_dividend_received_during_contract_cycle)'])/output_1['Strike Price'])+ ((output_1['interpolated_rate']/100)-(output_1['dividend_yield']/100) + (output_1['implied_volatility']/100)**2/2)*output_1['residual_maturity_yrs'])/((output_1['implied_volatility']/100)* np.sqrt(output_1['residual_maturity_yrs']))
output_1['d2'] = output_1['d1'] - (output_1['implied_volatility']/100) * np.sqrt(output_1['residual_maturity_yrs'])
output_1['call_option_premium'] = (output_1['underlying_spot_rate']-output_1['PV(total_dividend_received_during_contract_cycle)'])*np.exp(-output_1['dividend_yield']/100 * output_1['residual_maturity_yrs']) * norm.cdf(output_1['d1']) - output_1['Strike Price'] * np.exp(-output_1['interpolated_rate']/100 * output_1['residual_maturity_yrs']) * norm.cdf(output_1['d2'])
output_1['put_option_premium'] = (output_1['Strike Price'])*np.exp(-output_1['interpolated_rate']/100 * output_1['residual_maturity_yrs']) * norm.cdf(-output_1['d2']) - (output_1['underlying_spot_rate'] - output_1['PV(total_dividend_received_during_contract_cycle)']) * np.exp(-output_1['dividend_yield']/100 * output_1['residual_maturity_yrs']) * norm.cdf(-output_1['d1'])

In [15]:
output_1['mtm'] = np.where((output_1['option_type'] == 'C') & (output_1['Buy_Sell'] == 1),
                           output_1['call_option_premium'] * output_1['No of contracts'] * output_1['Lot Size'],
                           np.where((output_1['option_type'] == 'C') & (output_1['Buy_Sell'] == -1),
                           -output_1['call_option_premium'] * output_1['No of contracts'] * output_1['Lot Size'],
                           np.where((output_1['option_type'] == 'P') & (output_1['Buy_Sell'] == 1),
                           -output_1['put_option_premium'] * output_1['No of contracts'] * output_1['Lot Size'],
                           np.where((output_1['option_type'] == 'P') & (output_1['Buy_Sell'] == -1),
                           -output_1['put_option_premium'] * output_1['No of contracts'] * output_1['Lot Size'],
                           0))))

In [ ]:
output_1

In [ ]:
mapping_dict_exchange = dict(zip(zip(rates_exchange['date'], rates_exchange['base']), rates_exchange['exchange_rate']))
output_1['currency_transaction_domestic'] = output_1.apply(lambda row: mapping_dict_rates_implied_volatility.get((row['As_On_Date'], row['currency_domestic']), 1), axis=1)

output_1['mtm_domestic_ccy'] = output_1['mtm']* output_1['currency_transaction_domestic']

In [ ]:
mapping_dict_exchange = dict(zip(zip(rates_exchange['date'], rates_exchange['quote']), rates_exchange['exchange_rate']))
output_1['currency_transaction_reporting'] = output_1.apply(lambda row: mapping_dict_rates_implied_volatility.get((row['As_On_Date'], row['currency_reporting']), 1), axis=1)
output_1['mtm_reporting_ccy'] = output_1['mtm']* output_1['currency_transaction_reporting']

In [ ]:
output_1

In [ ]:
output_2 = pd.DataFrame()

output_2['Spot_Price'] = output_1['underlying_spot_rate'] - output_1['PV(total_dividend_received_during_contract_cycle)']
output_2['Exercise_Price'] = output_1['Strike Price']
output_2['Risk_Free_Rate'] = 0 #output_1['interpolated_rate']
output_2['Time_to_expiry'] = output_1['residual_maturity_yrs']
output_2['Volatility'] = output_1['implied_volatility']
output_2['Dividend_Yield'] = output_1['dividend_yield']
output_2['Delta_Call'] = np.exp(-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry'])* norm.cdf(output_1['d1'])
output_2['Gamma_Call'] = np.exp(-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry'])/(output_2['Spot_Price'] * (output_2['Volatility']/100) * np.sqrt(output_2['Time_to_expiry'])) * (np.exp(-(output_1['d1']**2)/2)/ np.sqrt(2*3.142))
output_2['Vega_Call'] = (output_2['Spot_Price'] * np.exp(-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry'])*np.sqrt(output_2['Time_to_expiry']) * (np.exp(-(output_1['d1']**2)/2)/ np.sqrt(2*3.142)))/100
output_2['Theta_call'] = ((-(output_2['Spot_Price'] * output_2['Volatility']/100 * np.exp (-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry'])/(2*np.sqrt(output_2['Time_to_expiry'])) * (np.exp(-((output_1['d1'])**2)/2)/(np.sqrt(2*3.142))))) - ((output_2['Risk_Free_Rate']/100)*output_2['Exercise_Price'] * np.exp(-output_2['Risk_Free_Rate']/100 * output_2['Time_to_expiry'])) * norm.cdf(output_1['d2']) + output_2['Dividend_Yield']/100 * output_2['Spot_Price'] * np.exp(-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry']) * norm.cdf(output_1['d1']))/365
output_2['Rho_Call'] = (output_2['Exercise_Price'] * output_2['Time_to_expiry'] * np.exp(-output_2['Risk_Free_Rate']/100 * output_2['Time_to_expiry'])*norm.cdf(output_1['d2']))/100

output_2['Delta_Put'] = np.exp(-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry'])* (norm.cdf(output_1['d1']-1))
output_2['Gamma_Put'] = output_2['Gamma_Call']
output_2['Vega_Put'] = output_2['Vega_Call']
output_2['Theta_call'] = ((-(output_2['Spot_Price'] * output_2['Volatility']/100 * np.exp (-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry'])/(2*np.sqrt(output_2['Time_to_expiry'])) * (np.exp(-((output_1['d1'])**2)/2)/(np.sqrt(2*3.142))))) - ((output_2['Risk_Free_Rate']/100)*output_2['Exercise_Price'] * np.exp(-output_2['Risk_Free_Rate']/100 * output_2['Time_to_expiry'])) * norm.cdf(output_1['d2']) + output_2['Dividend_Yield']/100 * output_2['Spot_Price'] * np.exp(-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry']) * norm.cdf(output_1['d1']))/365
output_2['Rho_Put'] = (output_2['Exercise_Price'] * output_2['Time_to_expiry'] * np.exp(-output_2['Risk_Free_Rate']/100 * output_2['Time_to_expiry'])*norm.cdf(output_1['d2']))/100

In [ ]:
output_2

In [ ]:
output_2['Theta_call'] = ((-(output_2['Spot_Price'] * output_2['Volatility']/100 * np.exp (-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry'])/(2*np.sqrt(output_2['Time_to_expiry'])) * (np.exp(-((output_1['d1'])**2)/2)/(np.sqrt(2*3.142))))) - 
                         ((output_2['Risk_Free_Rate']/100)*output_2['Exercise_Price'] * np.exp(-output_2['Risk_Free_Rate']/100 * output_2['Time_to_expiry'])) * norm.cdf(output_1['d2']) + output_2['Dividend_Yield']/100 * output_2['Spot_Price'] * 
                         np.exp(-output_2['Dividend_Yield']/100 * output_2['Time_to_expiry']) * norm.cdf(output_1['d1']))/365

In [ ]:
output_2